# Chapter 4: Compound Screening - Interactive Notebook

This notebook demonstrates code and concepts from [Chapter 4: Compound Screening](../chapters/chapter4-compound-screening.qmd) of the book.

You can run and modify the code cells below to explore compound screening hands-on.

## Install and Import Libraries

We'll use PubChemPy and ChEMBL WebResource Client for compound search. If not installed, uncomment the pip commands below.

In [1]:
#!pip install pubchempy chembl_webresource_client pandas

In [2]:
import sys
print(sys.executable)

/Users/justin/drug-discovery-ai/venv/bin/python3


In [3]:
# !pip install pubchempy chembl_webresource_client pandas
import pubchempy as pcp
from chembl_webresource_client.new_client import new_client
import pandas as pd

## Search PubChem for Aspirin

Let's search PubChem for aspirin and display its canonical SMILES.

In [4]:
compounds = pcp.get_compounds('aspirin', 'name')
for c in compounds:
    print(f"CID: {c.cid}, Canonical SMILES: {c.canonical_smiles}")

CID: 2244, Canonical SMILES: CC(=O)OC1=CC=CC=C1C(=O)O


/var/folders/db/dkqv3mq12fg2m2pqjzjx0_700000gn/T/ipykernel_47905/1900996233.py:3: PubChemPyDeprecationWarning: canonical_smiles is deprecated: Use connectivity_smiles instead
  print(f"CID: {c.cid}, Canonical SMILES: {c.canonical_smiles}")


## Search ChEMBL for Approved Drugs

Let's search ChEMBL for approved drugs and display their ChEMBL IDs and names.

In [5]:
molecule = new_client.molecule
approved_drugs = molecule.filter(max_phase=4)
for drug in approved_drugs[:5]:
    print(drug['molecule_chembl_id'], drug['pref_name'])

CHEMBL2 PRAZOSIN
CHEMBL3 NICOTINE
CHEMBL4 OFLOXACIN
CHEMBL5 NALIDIXIC ACID
CHEMBL6 INDOMETHACIN


## Link to Book Chapter

For more details and explanations, see [Chapter 4: Compound Screening](../chapters/chapter4-compound-screening.qmd) in the book.